In [1]:
from py2neo import Graph
from igraph import Graph as IGraph
from pprint import pprint

In [2]:
graph = Graph('bolt://127.0.0.1:7687', password='neo4jneo4j')

In [3]:
# 查詢台GG
query = '''
match (m:Stock{code:'2330'}) return m
'''
graph.run(query).data()

[{'m': Node('Stock', code='2330', community=0, market='上市', name='台積電', pagerank=0.0026358874823793387, stock_id='2330')}]

In [4]:
# Concept(概念股)種類的總數
query = '''
match (n:Concept) return count(distinct(n))
'''

graph.run(query)

count(distinct(n))
126


In [5]:
# GG有多少副總在其他公司也是董監事經理人或大股東
query = '''
MATCH (m:Stock{code:'2330'})<-[:employ_of{jobs:'副總經理'}]-(n:Person)-[e2:employ_of]->(q:Stock)
RETURN n, e2
'''
graph.run(query).data()

[{'n': Node('Person', community=1, name='李俊賢', pagerank=5.8791424382182885e-05, person_id='b9c64214b5e97d40aea34cfa75a390ac'),
  'e2': employ_of(Node('Person', community=1, name='李俊賢', pagerank=5.8791424382182885e-05, person_id='b9c64214b5e97d40aea34cfa75a390ac'), Node('Stock', code='2530', market='上市', name='華建', stock_id='2530'), jobs='副總經理', stock_num=203)},
 {'n': Node('Person', community=1, name='李俊賢', pagerank=5.8791424382182885e-05, person_id='b9c64214b5e97d40aea34cfa75a390ac'),
  'e2': employ_of(Node('Person', community=1, name='李俊賢', pagerank=5.8791424382182885e-05, person_id='b9c64214b5e97d40aea34cfa75a390ac'), Node('Stock', code='5201', market='上櫃', name='凱衛', stock_id='5201'), jobs='副總經理', stock_num=0, stock_ratio=0.0)},
 {'n': Node('Person', community=1, name='李俊賢', pagerank=5.8791424382182885e-05, person_id='b9c64214b5e97d40aea34cfa75a390ac'),
  'e2': employ_of(Node('Person', community=1, name='李俊賢', pagerank=5.8791424382182885e-05, person_id='b9c64214b5e97d40aea34cfa75a3

In [6]:
# GG有多少獨立董事
query = '''
MATCH (m:Stock{name:'台積電'})<-[:employ_of{jobs:'獨立董事'}]-(n:Person)
RETURN n, count(distinct(n))
'''
ig = IGraph.TupleList(graph.run(query), weights=True)
pprint([v for v in ig.vs])


[igraph.Vertex(<igraph.Graph object at 0x7f9c6840f940>, 0, {'name': Node('Person', community=1, name='摩西．蓋弗瑞', pagerank=1.8901045091885342e-05, person_id='f7e8a7073af3c121a18aba032a2b1a04')}),
 igraph.Vertex(<igraph.Graph object at 0x7f9c6840f940>, 1, {'name': 1}),
 igraph.Vertex(<igraph.Graph object at 0x7f9c6840f940>, 2, {'name': Node('Person', community=1, name='麥克。史賓林', pagerank=1.8901045091885342e-05, person_id='427f0333d16f224bedea4570e6cd9410')}),
 igraph.Vertex(<igraph.Graph object at 0x7f9c6840f940>, 3, {'name': Node('Person', community=1, name='陳國慈', pagerank=1.8901045091885342e-05, person_id='94417b1d3d9bec10492592510c3e3e40')}),
 igraph.Vertex(<igraph.Graph object at 0x7f9c6840f940>, 4, {'name': Node('Person', community=1, name='海英俊', pagerank=5.510402566637519e-05, person_id='6bcaa268b1b2ad70fc7a641edd421528')}),
 igraph.Vertex(<igraph.Graph object at 0x7f9c6840f940>, 5, {'name': Node('Person', community=1, name='拉斐爾‧萊夫', pagerank=1.8901045091885342e-05, person_id='6998f13

In [7]:
# 有多少'電腦及週邊設備業'也是'AI概念股'
query = '''
MATCH (:Concept{name:'AI人工智慧'})<-[:concept_of]-(m:Stock)-[:industry_of]->(:Industry{name:'電腦及週邊設備業'})
RETURN m, count(distinct(m))
'''
graph.run(query).data()

[{'m': Node('Stock', code='6414', community=0, market='上市', name='樺漢', pagerank=0.00112715267499116, stock_id='6414'),
  'count(distinct(m))': 1},
 {'m': Node('Stock', code='6166', community=0, market='上市', name='凌華', pagerank=0.0007455384110258626, stock_id='6166'),
  'count(distinct(m))': 1},
 {'m': Node('Stock', code='3231', community=0, market='上市', name='緯創', pagerank=0.002340426239425494, stock_id='3231'),
  'count(distinct(m))': 1},
 {'m': Node('Stock', code='2395', community=0, market='上市', name='研華', pagerank=0.0016078274695243253, stock_id='2395'),
  'count(distinct(m))': 1},
 {'m': Node('Stock', code='2382', community=0, market='上市', name='廣達', pagerank=0.002108673851452949, stock_id='2382'),
  'count(distinct(m))': 1},
 {'m': Node('Stock', code='2377', community=0, market='上市', name='微星', pagerank=0.0013429646826712258, stock_id='2377'),
  'count(distinct(m))': 1},
 {'m': Node('Stock', code='2357', community=0, market='上市', name='華碩', pagerank=0.0019307332802150363, stock_i

In [8]:
# 列出仁寶所有零持股的副總
query = '''
MATCH (:Stock{name:'仁寶'})<-[:employ_of{jobs:'副總經理', stock_num:0}]-(p1:Person)
RETURN p1, count(distinct(p1))
'''
graph.run(query).data()

[{'p1': Node('Person', community=1, name='黃士宏', pagerank=1.8256794678695572e-05, person_id='94d28f59154d23980b270c0d1dde4bd2'),
  'count(distinct(p1))': 1},
 {'p1': Node('Person', community=1, name='劉厚鈞', pagerank=1.8256794678695572e-05, person_id='4129087636a78ce3e7c9510cd11a9d51'),
  'count(distinct(p1))': 1},
 {'p1': Node('Person', community=1, name='劉平貴', pagerank=1.8256794678695572e-05, person_id='9b56a43be3f126da334b781d63da53bb'),
  'count(distinct(p1))': 1},
 {'p1': Node('Person', community=1, name='詹鵬弘', pagerank=1.8256794678695572e-05, person_id='124bc059485cb56ce5d3067dc03a6e6f'),
  'count(distinct(p1))': 1},
 {'p1': Node('Person', community=1, name='溫志偉', pagerank=1.8256794678695572e-05, person_id='e706756e2d1a336602b2e891279af81e'),
  'count(distinct(p1))': 1},
 {'p1': Node('Person', community=1, name='陳禧冠', pagerank=1.8256794678695572e-05, person_id='08881ed4de0d4e8a9f92584f1db92f1e'),
  'count(distinct(p1))': 1},
 {'p1': Node('Person', community=1, name='譚仲興', pagerank=1

In [9]:
# 列出仁寶副總所有持股的統計
query = '''
MATCH (:Stock{name:'仁寶'})<-[emp:employ_of{jobs:'副總經理'}]-(p1:Person)
WITH emp.stock_num AS num
RETURN min(num) AS min, max(num) AS max, avg(num) AS avg_characters, stdev(num) AS stdev
'''
graph.run(query).data()

[{'min': 0,
  'max': 10662,
  'avg_characters': 854.3714285714284,
  'stdev': 2155.553253476107}]

In [10]:
# 仁寶副總有多少比例零持股
query = '''
MATCH (:Stock{name:'仁寶'})<-[:employ_of{jobs:'副總經理', stock_num:0}]-(p1:Person)
MATCH (:Stock{name:'仁寶'})<-[:employ_of{jobs:'副總經理'}]-(p2:Person)
RETURN count(distinct(p1))*1.0/ count(distinct(p2)) as ratio
'''
graph.run(query)


ratio
0.4


In [11]:
# 仁寶副總有多少比例持股小於1000
query = '''
MATCH (:Stock{name:'仁寶'})<-[emp:employ_of{jobs:'副總經理'}]-(p1:Person)
MATCH (:Stock{name:'仁寶'})<-[:employ_of{jobs:'副總經理'}]-(p2:Person)
WHERE emp.stock_num < 1000
RETURN count(distinct(p1))*1.0/ count(distinct(p2)) as ratio
'''
graph.run(query)


ratio
0.8571428571428571


In [12]:
# 隔日沖券商 美林 買超張數超過1000張的股票
query = '''
MATCH (d:Dealer{name:'美林'})-[bs:buy_or_sell]->(s:Stock)
WHERE bs.amount>1000
RETURN *
'''
graph.run(query)

bs,d,s
(美林)-[:buy_or_sell {amount: 1647}]->(群創),"(_1084:Dealer {dealer_id: '1b1a8637036ae1e6a4f01618527e6914', name: '\u7f8e\u6797'})","(_11406:Stock {code: '3481', community: 0, market: '\u4e0a\u5e02', name: '\u7fa4\u5275', pagerank: 0.0017248332372997882, stock_id: '3481'})"
(美林)-[:buy_or_sell {amount: 3686}]->(緯創),"(_1084:Dealer {dealer_id: '1b1a8637036ae1e6a4f01618527e6914', name: '\u7f8e\u6797'})","(_11382:Stock {code: '3231', community: 0, market: '\u4e0a\u5e02', name: '\u7def\u5275', pagerank: 0.002340426239425494, stock_id: '3231'})"
(美林)-[:buy_or_sell {amount: 2163}]->(第一金),"(_1084:Dealer {dealer_id: '1b1a8637036ae1e6a4f01618527e6914', name: '\u7f8e\u6797'})","(_11302:Stock {code: '2892', community: 21, market: '\u4e0a\u5e02', name: '\u7b2c\u4e00\u91d1', pagerank: 0.0004362672117722965, stock_id: '2892'})"


In [13]:
# GG跟發哥在概念股上的最短路徑
query = '''
MATCH (a:Stock {code:'2330'}), (b:Stock {code:'2454'})
MATCH p=allShortestPaths((a)-[:concept_of*]-(b))
WITH [node IN nodes(p) where node:Concept | node.name] AS concept
RETURN concept 
'''
graph.run(query).data()

[{'concept': ['華為']},
 {'concept': ['AI人工智慧']},
 {'concept': ['3D感測']},
 {'concept': ['車聯網']},
 {'concept': ['手機']},
 {'concept': ['網通']},
 {'concept': ['山寨手機']},
 {'concept': ['APPLE概念']},
 {'concept': ['3D技術']},
 {'concept': ['iTV']},
 {'concept': ['車用電子相關']},
 {'concept': ['5G']}]

In [14]:
# 查詢含有最多degree(不考慮方向以及關聯類別)的前10個Stock
# 彰銀主要靠董事成員夠多
query = '''
MATCH (s:Stock)
RETURN s.name AS stock, apoc.node.degree(s) AS degree 
ORDER BY degree DESC LIMIT 10
'''

graph.run(query).data()

[{'stock': '彰銀', 'degree': 268},
 {'stock': '臺企銀', 'degree': 221},
 {'stock': '安泰銀', 'degree': 197},
 {'stock': '凌群', 'degree': 190},
 {'stock': '遠東銀', 'degree': 188},
 {'stock': '聯邦銀', 'degree': 175},
 {'stock': '開發金', 'degree': 163},
 {'stock': '上海商銀', 'degree': 161},
 {'stock': '台中銀', 'degree': 161},
 {'stock': '中華電', 'degree': 147}]

In [15]:
# 查詢跟最多股票有關係的前35位董事
# 涂水城是大股東
query = '''
MATCH (p:Person)
RETURN p.name AS person, apoc.node.degree(p) AS degree 
ORDER BY degree DESC LIMIT 35
'''

graph.run(query).data()

[{'person': '莊永順', 'degree': 11},
 {'person': '李俊德', 'degree': 9},
 {'person': '焦佑衡', 'degree': 9},
 {'person': '王文淵', 'degree': 8},
 {'person': '姚德彰', 'degree': 7},
 {'person': '陳金龍', 'degree': 7},
 {'person': '陳建志', 'degree': 7},
 {'person': '林志隆', 'degree': 7},
 {'person': '陳俊成', 'degree': 6},
 {'person': '陳進財', 'degree': 6},
 {'person': '胡秋江', 'degree': 6},
 {'person': '徐旭東', 'degree': 6},
 {'person': '焦佑鈞', 'degree': 6},
 {'person': '陳冠華', 'degree': 6},
 {'person': '統一企業(股)-羅智先', 'degree': 6},
 {'person': '王俊傑', 'degree': 6},
 {'person': '鄭更義', 'degree': 6},
 {'person': '陳建州', 'degree': 6},
 {'person': '林志峰', 'degree': 6},
 {'person': '王光祥', 'degree': 6},
 {'person': '汪雅康', 'degree': 6},
 {'person': '陳雍之', 'degree': 6},
 {'person': '李英珍', 'degree': 6},
 {'person': '蔡政憲', 'degree': 6},
 {'person': '郭淑珍', 'degree': 5},
 {'person': '席家宜', 'degree': 5},
 {'person': '陳怡君', 'degree': 5},
 {'person': '林建宏', 'degree': 5},
 {'person': '陳信宏', 'degree': 5},
 {'person': '陳淑玲', 'degree': 5},
 

In [16]:
# 查詢總持股前35最多的董事
query = '''
MATCH (p:Person)-[r:employ_of]-(:Stock)
RETURN p.name AS person, sum(r.stock_num) as stockNum
ORDER BY stockNum DESC LIMIT 35
'''

graph.run(query).data()

[{'person': '財政部-許國郎', 'stockNum': 3651323},
 {'person': '財政部-鄧延達', 'stockNum': 3651323},
 {'person': '財政部-雷仲達', 'stockNum': 3651323},
 {'person': '財政部-陳美足', 'stockNum': 3651323},
 {'person': '財政部-江瑞堂', 'stockNum': 3651323},
 {'person': '財政部-周惠美', 'stockNum': 3651323},
 {'person': '經濟部(股)-劉明忠', 'stockNum': 3486011},
 {'person': '台灣化學纖維(股)-王文淵', 'stockNum': 3417801},
 {'person': '經濟部(股)-翁朝棟', 'stockNum': 3154709},
 {'person': '經濟部(股)-曾文生', 'stockNum': 3154709},
 {'person': '臺灣銀行(股)-王昭文', 'stockNum': 2896526},
 {'person': '臺灣銀行(股)-鄭士卿', 'stockNum': 2896526},
 {'person': '臺灣銀行(股)-蔡偉德', 'stockNum': 2896526},
 {'person': '臺灣銀行(股)-王文杰', 'stockNum': 2896526},
 {'person': '臺灣銀行(股)-王安邦', 'stockNum': 2896526},
 {'person': '萬寶開發(股)', 'stockNum': 2788405},
 {'person': '交通部-蔡秀涓', 'stockNum': 2737719},
 {'person': '交通部-陳信宏', 'stockNum': 2737719},
 {'person': '交通部-謝繼茂', 'stockNum': 2737719},
 {'person': '交通部-曾世宏', 'stockNum': 2737719},
 {'person': '交通部-胡湘麟', 'stockNum': 2737719},
 {'person': '交通部-張信一

In [17]:
%%time
# PageRank: The size of each node is proportional to the number and size of the other nodes pointing to it in the network. 
# 使用pagerank篩選在所有概念股類別當中，參與這些概念股前20多的股票
query = '''
MATCH (s:Stock)-[r:concept_of]->(c:Concept)
RETURN s.name, c.name
'''
ig = IGraph.TupleList(graph.run(query), weights=True)

pg = ig.pagerank()
pgvs = []
for p in zip(ig.vs, pg):
    pgvs.append({"name": p[0]["name"], "pg": p[1]})

write_clusters_query = '''
UNWIND $nodes AS n
MATCH (s:Stock) WHERE s.name = n.name
SET s.pagerank = n.pg
'''
graph.run(write_clusters_query, nodes=pgvs)


query = '''
MATCH (s:Stock)--(st:StockType{name:'股票'})
WHERE s.pagerank<>'none'
RETURN s.name AS name, s.pagerank AS pagerank ORDER BY pagerank DESC LIMIT 20
'''
graph.run(query).data()

CPU times: user 222 ms, sys: 0 ns, total: 222 ms
Wall time: 14 s


[{'name': '台達電', 'pagerank': 0.004449776383402385},
 {'name': '鴻海', 'pagerank': 0.0043005834611744885},
 {'name': '三星', 'pagerank': 0.003135367787337485},
 {'name': '瑞昱', 'pagerank': 0.0027106985889843197},
 {'name': '台積電', 'pagerank': 0.002635887482379148},
 {'name': '聯發科', 'pagerank': 0.002486829721305543},
 {'name': '緯創', 'pagerank': 0.0023404262394254066},
 {'name': '正崴', 'pagerank': 0.002323694754789855},
 {'name': '原相', 'pagerank': 0.0022495755471836046},
 {'name': '英業達', 'pagerank': 0.0021179553042175226},
 {'name': '廣達', 'pagerank': 0.00210867385145294},
 {'name': '仁寶', 'pagerank': 0.0020391599186539103},
 {'name': '偉詮電', 'pagerank': 0.001976949002979433},
 {'name': '和碩', 'pagerank': 0.0019500213697815926},
 {'name': '華碩', 'pagerank': 0.0019307332802153113},
 {'name': '鴻準', 'pagerank': 0.0019036205871657316},
 {'name': '東元', 'pagerank': 0.0018812958580418059},
 {'name': '大立光', 'pagerank': 0.0018679439176872003},
 {'name': '日月光投控', 'pagerank': 0.0017697817854847129},
 {'name': '

In [18]:
%%time
# 使用各檔股票的概念股來做各檔股票的community detection

clusters = IGraph.community_walktrap(ig, steps=3).as_clustering()

nodes = [{"name": node["name"]} for node in ig.vs]
for node in nodes:
    idx = ig.vs.find(name=node["name"]).index
    node["community"] = clusters.membership[idx]

write_clusters_query = '''
UNWIND $nodes AS n
MATCH (s:Stock) WHERE s.name = n.name
SET s.community = toInteger(n.community)
'''

graph.run(write_clusters_query, nodes=nodes)

query = '''
MATCH (s:Stock)
WITH s.community AS cluster, collect(s.name) AS members
WHERE cluster<>'none'
RETURN cluster, members ORDER BY cluster ASC
'''

graph.run(query).data()[0]


CPU times: user 94.5 ms, sys: 1.05 ms, total: 95.5 ms
Wall time: 11.1 s


{'cluster': 0,
 'members': ['僑威',
  '聯亞',
  '笙泉',
  '大綜',
  '璟德',
  '波若威',
  '亞信',
  '順達',
  '優群',
  '原相',
  '金麗科',
  '光環',
  '鑫創',
  '威剛',
  '東碩',
  '宇環',
  '微端',
  '尼克森',
  '建舜電',
  '雙鴻',
  '尚立',
  '上詮',
  '崇越電',
  '旭軟',
  '由田',
  '致振',
  '力致',
  '昇達科',
  '矽瑪',
  '鴻翊',
  '迎輝',
  '力旺',
  '兆利',
  '禾瑞亞',
  '神準',
  '牧德',
  '泓格',
  '磐儀',
  '鼎翰',
  '安可',
  '富晶通',
  '新鉅科',
  '艾恩特',
  '精聯',
  '榮昌',
  '湧德',
  '營邦',
  '鐿鈦',
  '桓達',
  '永捷',
  '熒茂',
  '聯光通',
  '前鼎',
  '新復興',
  '友輝',
  '亞電',
  '緯軟',
  '譜瑞-KY',
  '華星光',
  '新鼎',
  '智晶',
  '笙科',
  '信驊',
  '台林',
  '協益',
  '中光電',
  '應華',
  '中菲',
  '宣德',
  '通泰',
  '松普',
  '同亨',
  '創惟',
  '新普',
  '上奇',
  '萬旭',
  '茂達',
  '耕興',
  '頎邦',
  '欣技',
  '華電網',
  '亞通',
  '幃翔',
  '萬潤',
  '廣明',
  '萬泰科',
  '系微',
  '旺玖',
  '立端',
  '久元',
  '普萊德',
  '台燿',
  '良維',
  '沛亨',
  '晶焱',
  '統新',
  '迅得',
  '神盾',
  '宇智',
  '互動',
  '創威',
  '宏觀',
  '光菱',
  '榮群',
  '晶采',
  '廣積',
  '凱碩',
  '元太',
  '宏捷科',
  '華宏',
  '泰藝',
  '群聯',
  '商之器',
  '華研',
  '信錦',
  '聲寶',
  '華榮',
  '三洋電',
  '川湖',

In [19]:
%%time
# 使用pagerank篩選在所有股票當中，參與這些股票前20多的董事

query = '''
MATCH (p:Person)-[r:employ_of]->(s:Stock)
RETURN p.name, s.name
'''
ig = IGraph.TupleList(graph.run(query), weights=True)

pg = ig.pagerank()
pgvs = []
for p in zip(ig.vs, pg):
    pgvs.append({"name": p[0]["name"], "pg": p[1]})

write_clusters_query = '''
UNWIND $nodes AS n
MATCH (p:Person) WHERE p.name = n.name
SET p.pagerank = n.pg
'''
graph.run(write_clusters_query, nodes=pgvs)


query = '''
MATCH (p:Person)
RETURN p.name AS name, p.pagerank AS pagerank ORDER BY pagerank DESC LIMIT 20
'''
graph.run(query).data()



CPU times: user 904 ms, sys: 13.7 ms, total: 918 ms
Wall time: 8min 45s


[{'name': '莊永順', 'pagerank': 0.00013642862956526018},
 {'name': '李俊德', 'pagerank': 0.00012217924597266868},
 {'name': '焦佑衡', 'pagerank': 0.00011283704399921802},
 {'name': '王文淵', 'pagerank': 0.00010226818288853262},
 {'name': '林志隆', 'pagerank': 9.71753667194982e-05},
 {'name': '姚德彰', 'pagerank': 9.699060770552947e-05},
 {'name': '陳金龍', 'pagerank': 9.602838124622436e-05},
 {'name': '陳建志', 'pagerank': 9.147055518385646e-05},
 {'name': '陳俊成', 'pagerank': 8.491468337679678e-05},
 {'name': '汪雅康', 'pagerank': 8.394703479567012e-05},
 {'name': '王俊傑', 'pagerank': 8.34041199849523e-05},
 {'name': '鄭更義', 'pagerank': 8.30350869266328e-05},
 {'name': '林志峰', 'pagerank': 8.299302447762641e-05},
 {'name': '陳冠華', 'pagerank': 8.276717381662142e-05},
 {'name': '陳建州', 'pagerank': 8.274753451978074e-05},
 {'name': '陳進財', 'pagerank': 8.155889162700944e-05},
 {'name': '胡秋江', 'pagerank': 8.152911008675036e-05},
 {'name': '徐旭東', 'pagerank': 8.078803264253219e-05},
 {'name': '蔡政憲', 'pagerank': 8.04818021861452

In [20]:
len(pgvs)

27908

In [21]:
%%time
# 使用各檔股票來做董事的community detection
# 看看是否有哪些董事是同一群的
# 也就是共同都是某些股票的董事/大股東
# 理論上只有少數人身兼多家公司董事/大股東


clusters = IGraph.community_walktrap(ig, steps=2).as_clustering()

nodes = [{"name": node["name"]} for node in ig.vs]
for node in nodes:
    idx = ig.vs.find(name=node["name"]).index
    node["community"] = clusters.membership[idx]

write_clusters_query = '''
UNWIND $nodes AS n
MATCH (p:Person) WHERE p.name = n.name
SET p.community = toInteger(n.community)
'''

graph.run(write_clusters_query, nodes=nodes)

query = '''
MATCH (p:Person)
WITH p.community AS cluster, collect(p.name) AS members
WHERE cluster<>'none'
RETURN cluster, members ORDER BY cluster ASC
'''

graph.run(query).data()[0]

CPU times: user 22.9 s, sys: 81.4 ms, total: 23 s
Wall time: 6min 39s


{'cluster': 0,
 'members': ['徐維宏',
  '邱紹齊',
  '魏恒巍',
  '茂元國際(股)-羅任芬',
  '李安宗',
  '李建裕',
  '羅任芬',
  '葉明球',
  '韓千山',
  '黃大中',
  '瓏德國際事業股份-黃大中',
  '林信鴻',
  '鍾亮宏']}

In [22]:
person_clusters = graph.run(query).data()
len(person_clusters)

906

In [23]:
write_stock_weight_query = '''
MATCH (p:Person)-[r:employ_of]->(s:Stock)
RETURN p.name as person_name, r.stock_num as stock_num, s.name as stock_name
'''

data = graph.run(write_stock_weight_query).data()


In [29]:
stock_query = '''
MATCH (p:Person)-[r:employ_of]->(s:Stock)
RETURN DISTINCT s.name as stock_name
'''
from collections import defaultdict

total_stock_nums = defaultdict(int)
stock_name = graph.run(stock_query).data()
for person in data:
    total_stock_nums[person['stock_name']] += person['stock_num']
for person in data:
    try:
        person['stock_ratio'] = person['stock_num']/total_stock_nums[person['stock_name']]
    except ZeroDivisionError:
        person['stock_ratio'] = 0


In [30]:
total_stock_nums[person['stock_name']]

68287

In [31]:
total_stock_nums['台積電']  

1737191

In [32]:
%%time
## Person和Stock關聯employ_of線的粗細，用股東總持股當分母，董事持股數當分子

write_stock_weight_query = '''
UNWIND $nodes AS n
MATCH (p:Person)-[r:employ_of]->(s:Stock)
WHERE p.name = n.person_name AND s.name = n.stock_name
SET r.stock_ratio = n.stock_ratio
'''
graph.run(write_stock_weight_query, nodes=data).data()

CPU times: user 256 ms, sys: 4.04 ms, total: 260 ms
Wall time: 4min 48s


[]

In [33]:
# 畫出半導體業的股票與董事之間的關係，限制1000個entity
query = '''
MATCH (s:Stock)-[:industry_of]->(Industry{name:'半導體業'})
WITH s
MATCH (p:Person)-[r:employ_of]->(s) 
RETURN * LIMIT 1000
'''
graph.run(query).to_table()[:5]

[(Node('Person', community=75, name='葉如穎', pagerank=1.9142063890838076e-05, person_id='2d61aba76f61235a4f3bdcd80383f0e5'),
  employ_of(Node('Person', community=75, name='葉如穎', pagerank=1.9142063890838076e-05, person_id='2d61aba76f61235a4f3bdcd80383f0e5'), Node('Stock', code='3073', market='上櫃', name='凱柏實業', stock_id='3073'), jobs='財務主管', stock_num=0, stock_ratio=0.0),
  Node('Stock', code='3073', market='上櫃', name='凱柏實業', stock_id='3073')),
 (Node('Person', community=75, name='朱乃文', pagerank=1.9142063890838076e-05, person_id='3486f8db30e753af18d157b1e76f76cc'),
  employ_of(Node('Person', community=75, name='朱乃文', pagerank=1.9142063890838076e-05, person_id='3486f8db30e753af18d157b1e76f76cc'), Node('Stock', code='3073', market='上櫃', name='凱柏實業', stock_id='3073'), jobs='副理', stock_num=0, stock_ratio=0.0),
  Node('Stock', code='3073', market='上櫃', name='凱柏實業', stock_id='3073')),
 (Node('Person', community=75, name='黃小玲', pagerank=1.9142063890838076e-05, person_id='f419adeadbee0bed49bd1cca8